### Import libraries.

In [1]:
import requests
import numpy as np

import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import random

import json
from pandas.io.json import json_normalize

from IPython.display import Image
from IPython.core.display import HTML

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim

!conda install -c conda-forge folium=0.5.0 --yes
import folium

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2020.6.20  |       hecda079_0         145 KB  conda-forge
    openssl-1.1.1g             |       h516909a_0         2.1 MB  conda-forge
    geopy-2.0.0                |     pyh9f0ad1d_0          63 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    certifi-2020.6.20          |   py36h9f0ad1d_0         151 KB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0          conda-forge
    geopy:           

### Validate Foursquare credentials.

In [2]:
CLIENT_ID = 'DPR2TRJDTRBYCTN4YIRLNJ4F3GRM3IHAWXWCB4MQGXDU2H5E'
CLIENT_SECRET = 'ZVGOPN1OCEFCGB5RDDUQXBGE11AXDT2OHB0VNGFUAZUO3COZ'
VERSION = '20200713'
LIMIT = 30
print('Your credentials:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET: ' + CLIENT_SECRET)

Your credentials:
CLIENT_ID: DPR2TRJDTRBYCTN4YIRLNJ4F3GRM3IHAWXWCB4MQGXDU2H5E
CLIENT_SECRET: ZVGOPN1OCEFCGB5RDDUQXBGE11AXDT2OHB0VNGFUAZUO3COZ


### Address of potential property in Downtown Waco off I-35.

In [3]:
address = '508 Franklin Ave, Waco, TX'

geolocator = Nominatim(user_agent="Texas_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geographical coordiate of Waco are {}, {}.'.format(latitude, longitude))

The geographical coordiate of Waco are 31.555734857142856, -97.13097791836735.


### '508 Franklin Ave, Waco, TX' mark on Folium map of Waco.

In [4]:
map_waco = folium.Map(location=[latitude, longitude], zoom_start=14)

folium.features.CircleMarker(
    [latitude, longitude],
    radius=8,
    color='red',
    popup='Downtown Waco Location',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(map_waco)
 
map_waco

### First search for 'BBQ' in Downtown Waco.

In [5]:
search_query = 'bbq'
radius = 500
print(search_query + '.... OK!')

bbq.... OK!


In [6]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=DPR2TRJDTRBYCTN4YIRLNJ4F3GRM3IHAWXWCB4MQGXDU2H5E&client_secret=ZVGOPN1OCEFCGB5RDDUQXBGE11AXDT2OHB0VNGFUAZUO3COZ&ll=31.555734857142856,-97.13097791836735&v=20200713&query=bbq&radius=500&limit=30'

In [7]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5f0c74e6a10064683eb71228'},
 'response': {'venues': [{'id': '560d60e8498ed394e9de762e',
    'name': 'Honky Tonk Kid BBQ',
    'location': {'address': '112 Mary Avenue',
     'crossStreet': 'University Drive',
     'lat': 31.557973,
     'lng': -97.126022,
     'labeledLatLngs': [{'label': 'display',
       'lat': 31.557973,
       'lng': -97.126022}],
     'distance': 532,
     'postalCode': '76701',
     'cc': 'US',
     'city': 'Waco',
     'state': 'TX',
     'country': 'United States',
     'formattedAddress': ['112 Mary Avenue (University Drive)',
      'Waco, TX 76701',
      'United States']},
    'categories': [{'id': '4bf58dd8d48988d1df931735',
      'name': 'BBQ Joint',
      'pluralName': 'BBQ Joints',
      'shortName': 'BBQ',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/bbqalt_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1594651938',
    'hasPerk': False}]}}

### Get relevant part of JSON file and transform into pandas dataframe. 

In [8]:
venues = results['response']['venues']

In [9]:
dataframe = json_normalize(venues)
dataframe.head()

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.postalCode,location.state,name,referralId
0,"[{'id': '4bf58dd8d48988d1df931735', 'name': 'B...",False,560d60e8498ed394e9de762e,112 Mary Avenue,US,Waco,United States,University Drive,532,"[112 Mary Avenue (University Drive), Waco, TX ...","[{'label': 'display', 'lat': 31.557973, 'lng':...",31.557973,-97.126022,76701,TX,Honky Tonk Kid BBQ,v-1594651938


### Define information of interest.

In [10]:
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
    
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,Honky Tonk Kid BBQ,BBQ Joint,112 Mary Avenue,US,Waco,United States,University Drive,532,"[112 Mary Avenue (University Drive), Waco, TX ...","[{'label': 'display', 'lat': 31.557973, 'lng':...",31.557973,-97.126022,76701,TX,560d60e8498ed394e9de762e


In [11]:
dataframe_filtered.name

0    Honky Tonk Kid BBQ
Name: name, dtype: object

### Location of "Honky Tonk Kid BBQ" in proximity to the Downtown location. 

In [12]:
map_waco = folium.Map(location=[latitude, longitude], zoom_start=14)

folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='Downtown Waco Location',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(map_waco)

for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.features.CircleMarker(
    [lat, lng],
    radius=5,
    color='blue',
    popup=label,
    fill = True,
    fill_color='blue',
    fill_opacity=0.6
    ).add_to(map_waco)
    
map_waco

### Let's explore "Honky Tonk Kid BBQ."

In [13]:
venue_id = '560d60e8498ed394e9de762e'
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)
url

'https://api.foursquare.com/v2/venues/560d60e8498ed394e9de762e?client_id=DPR2TRJDTRBYCTN4YIRLNJ4F3GRM3IHAWXWCB4MQGXDU2H5E&client_secret=ZVGOPN1OCEFCGB5RDDUQXBGE11AXDT2OHB0VNGFUAZUO3COZ&v=20200713'

In [14]:
result = requests.get(url).json()
print(result['response']['venue'].keys())
result['response']['venue']

dict_keys(['id', 'name', 'contact', 'location', 'canonicalUrl', 'categories', 'verified', 'stats', 'price', 'likes', 'dislike', 'ok', 'allowMenuUrlEdit', 'beenHere', 'specials', 'photos', 'reasons', 'description', 'hereNow', 'createdAt', 'tips', 'shortUrl', 'timeZone', 'listed', 'hours', 'seasonalHours', 'defaultHours', 'pageUpdates', 'inbox', 'attributes', 'bestPhoto'])


{'id': '560d60e8498ed394e9de762e',
 'name': 'Honky Tonk Kid BBQ',
 'contact': {'phone': '2547492368',
  'formattedPhone': '(254) 749-2368',
  'twitter': 'honkytonkkidbbq',
  'instagram': 'honkytonkkidbbq'},
 'location': {'address': '112 Mary Avenue',
  'crossStreet': 'University Drive',
  'lat': 31.557973,
  'lng': -97.126022,
  'labeledLatLngs': [{'label': 'display',
    'lat': 31.557973,
    'lng': -97.126022}],
  'postalCode': '76701',
  'cc': 'US',
  'city': 'Waco',
  'state': 'TX',
  'country': 'United States',
  'formattedAddress': ['112 Mary Avenue (University Drive)',
   'Waco, TX 76701',
   'United States']},
 'canonicalUrl': 'https://foursquare.com/v/honky-tonk-kid-bbq/560d60e8498ed394e9de762e',
 'categories': [{'id': '4bf58dd8d48988d1df931735',
   'name': 'BBQ Joint',
   'pluralName': 'BBQ Joints',
   'shortName': 'BBQ',
   'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/bbqalt_',
    'suffix': '.png'},
   'primary': True}],
 'verified': False,
 'stats': {'ti

In [15]:
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

This venue has not been rated yet.


### Get number of tips.

In [16]:
result['response']['venue']['tips']['count']

4

In [17]:
limit = 15
url = 'https://api.foursquare.com/v2/venues/{}/tips?client_id={}&client_secret={}&v={}&limit={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION,limit)
url

'https://api.foursquare.com/v2/venues/560d60e8498ed394e9de762e/tips?client_id=DPR2TRJDTRBYCTN4YIRLNJ4F3GRM3IHAWXWCB4MQGXDU2H5E&client_secret=ZVGOPN1OCEFCGB5RDDUQXBGE11AXDT2OHB0VNGFUAZUO3COZ&v=20200713&limit=15'

In [18]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5f0c74e939be673584fe3c23'},
 'response': {'tips': {'count': 4,
   'items': [{'id': '58cc3d39c0c89b181c80675f',
     'createdAt': 1489780025,
     'text': 'Tasty brisket, tender pork, delicious ribs...',
     'type': 'user',
     'canonicalUrl': 'https://foursquare.com/item/58cc3d39c0c89b181c80675f',
     'lang': 'en',
     'likes': {'count': 0, 'groups': []},
     'logView': True,
     'agreeCount': 3,
     'disagreeCount': 0,
     'todo': {'count': 0},
     'user': {'id': '133690',
      'firstName': 'Mike',
      'lastName': 'D',
      'photo': {'prefix': 'https://fastly.4sqi.net/img/user/',
       'suffix': '/XGM1ZZXR0VSNAHQK.jpg'}},
     'authorInteractionType': 'liked'}]}}}

In [19]:
tips = results['response']['tips']['items']

tip = results['response']['tips']['items'][0]
tip.keys()

dict_keys(['id', 'createdAt', 'text', 'type', 'canonicalUrl', 'lang', 'likes', 'logView', 'agreeCount', 'disagreeCount', 'todo', 'user', 'authorInteractionType'])

In [20]:
pd.set_option('display.max_colwidth', -1)

tips_df = json_normalize(tips)

filtered_columns = ['text', 'agreeCount', 'disagreeCount', 'id', 'user.firstName', 'user.lastName', 'user.gender', 'user.id']
tips_filtered = tips_df.loc[:, filtered_columns]

tips_filtered

/opt/conda/envs/Python36/lib/python3.6/site-packages/pandas/core/indexing.py:1494: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)


,text,agreeCount,disagreeCount,id,user.firstName,user.lastName,user.gender,user.id
0,"Tasty brisket, tender pork, delicious ribs...",3,0,58cc3d39c0c89b181c80675f,Mike,D,NaN,133690


### Second search for 'Barbeque' in Downtown Waco.

In [21]:
search_query = 'barbeque'
radius = 500
print(search_query + '.... OK!')

barbeque.... OK!


In [22]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=DPR2TRJDTRBYCTN4YIRLNJ4F3GRM3IHAWXWCB4MQGXDU2H5E&client_secret=ZVGOPN1OCEFCGB5RDDUQXBGE11AXDT2OHB0VNGFUAZUO3COZ&ll=31.555734857142856,-97.13097791836735&v=20200713&query=barbeque&radius=500&limit=30'

In [23]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5f0c7575cedc4c5633eed78d'},
 'response': {'venues': [{'id': '587a693e15fb436da137bce7',
    'name': 'Guess Family Barbeque',
    'location': {'address': '325 S 6th',
     'lat': 31.553531,
     'lng': -97.130228,
     'labeledLatLngs': [{'label': 'display',
       'lat': 31.553531,
       'lng': -97.130228}],
     'distance': 255,
     'postalCode': '76701',
     'cc': 'US',
     'city': 'Waco',
     'state': 'TX',
     'country': 'United States',
     'formattedAddress': ['325 S 6th', 'Waco, TX 76701', 'United States']},
    'categories': [{'id': '4bf58dd8d48988d1df931735',
      'name': 'BBQ Joint',
      'pluralName': 'BBQ Joints',
      'shortName': 'BBQ',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/bbqalt_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1594652054',
    'hasPerk': False}]}}

### Get relevant part of JSON file and transform into pandas dataframe. 

In [24]:
venues = results['response']['venues']

In [25]:
dataframe = json_normalize(venues)
dataframe.head()

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.postalCode,location.state,name,referralId
0,"[{'id': '4bf58dd8d48988d1df931735', 'name': 'BBQ Joint', 'pluralName': 'BBQ Joints', 'shortName': 'BBQ', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/bbqalt_', 'suffix': '.png'}, 'primary': True}]",False,587a693e15fb436da137bce7,325 S 6th,US,Waco,United States,255,"[325 S 6th, Waco, TX 76701, United States]","[{'label': 'display', 'lat': 31.553531, 'lng': -97.130228}]",31.553531,-97.130228,76701,TX,Guess Family Barbeque,v-1594652054


### Define information of interest.

In [26]:
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
    
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,cc,city,country,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,Guess Family Barbeque,BBQ Joint,325 S 6th,US,Waco,United States,255,"[325 S 6th, Waco, TX 76701, United States]","[{'label': 'display', 'lat': 31.553531, 'lng': -97.130228}]",31.553531,-97.130228,76701,TX,587a693e15fb436da137bce7


In [27]:
dataframe_filtered.name

0    Guess Family Barbeque
Name: name, dtype: object

### Location of "Guess Family Barbeque" in proximity to the Downtown location. 

In [28]:
map_waco = folium.Map(location=[latitude, longitude], zoom_start=14)

folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='Downtown Waco Location',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(map_waco)


for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.features.CircleMarker(
    [lat, lng],
    radius=5,
    color='blue',
    popup=label,
    fill = True,
    fill_color='blue',
    fill_opacity=0.6
    ).add_to(map_waco)
    
    
map_waco

### Let's explore "Guess Family Barbeque".

In [29]:
venue_id = '587a693e15fb436da137bce7'
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)
url

'https://api.foursquare.com/v2/venues/587a693e15fb436da137bce7?client_id=DPR2TRJDTRBYCTN4YIRLNJ4F3GRM3IHAWXWCB4MQGXDU2H5E&client_secret=ZVGOPN1OCEFCGB5RDDUQXBGE11AXDT2OHB0VNGFUAZUO3COZ&v=20200713'

In [30]:
result = requests.get(url).json()
print(result['response']['venue'].keys())
result['response']['venue']

dict_keys(['id', 'name', 'contact', 'location', 'canonicalUrl', 'categories', 'verified', 'stats', 'url', 'price', 'likes', 'dislike', 'ok', 'allowMenuUrlEdit', 'beenHere', 'specials', 'photos', 'reasons', 'hereNow', 'createdAt', 'tips', 'shortUrl', 'timeZone', 'listed', 'hours', 'seasonalHours', 'defaultHours', 'pageUpdates', 'inbox', 'attributes', 'bestPhoto'])


{'id': '587a693e15fb436da137bce7',
 'name': 'Guess Family Barbeque',
 'contact': {'phone': '2543017996',
  'formattedPhone': '(254) 301-7996',
  'twitter': 'guessfamilybbq'},
 'location': {'address': '325 S 6th',
  'lat': 31.553531,
  'lng': -97.130228,
  'labeledLatLngs': [{'label': 'display',
    'lat': 31.553531,
    'lng': -97.130228}],
  'postalCode': '76701',
  'cc': 'US',
  'city': 'Waco',
  'state': 'TX',
  'country': 'United States',
  'formattedAddress': ['325 S 6th', 'Waco, TX 76701', 'United States']},
 'canonicalUrl': 'https://foursquare.com/v/guess-family-barbeque/587a693e15fb436da137bce7',
 'categories': [{'id': '4bf58dd8d48988d1df931735',
   'name': 'BBQ Joint',
   'pluralName': 'BBQ Joints',
   'shortName': 'BBQ',
   'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/bbqalt_',
    'suffix': '.png'},
   'primary': True}],
 'verified': False,
 'stats': {'tipCount': 2},
 'url': 'http://guessfamilybbq.com',
 'price': {'tier': 2, 'message': 'Moderate', 'currenc

In [31]:
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

This venue has not been rated yet.


### Get number of tips.

In [32]:
result['response']['venue']['tips']['count']

2

In [33]:
limit = 15
url = 'https://api.foursquare.com/v2/venues/{}/tips?client_id={}&client_secret={}&v={}&limit={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION,limit)
url

'https://api.foursquare.com/v2/venues/587a693e15fb436da137bce7/tips?client_id=DPR2TRJDTRBYCTN4YIRLNJ4F3GRM3IHAWXWCB4MQGXDU2H5E&client_secret=ZVGOPN1OCEFCGB5RDDUQXBGE11AXDT2OHB0VNGFUAZUO3COZ&v=20200713&limit=15'

In [34]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5f0c74380ab6c4151b457937'},
 'response': {'tips': {'count': 2,
   'items': [{'id': '5c55d5c3418686002cfac954',
     'createdAt': 1549129155,
     'text': 'So amazing, try the cheese grits!',
     'type': 'user',
     'canonicalUrl': 'https://foursquare.com/item/5c55d5c3418686002cfac954',
     'lang': 'en',
     'likes': {'count': 0, 'groups': []},
     'logView': True,
     'agreeCount': 0,
     'disagreeCount': 0,
     'todo': {'count': 0},
     'user': {'id': '1939393',
      'firstName': 'Caryn',
      'lastName': 'B',
      'photo': {'prefix': 'https://fastly.4sqi.net/img/user/',
       'suffix': '/1CKMDU0VMIBJX2NF.jpg'}},
     'authorInteractionType': 'liked'}]}}}

In [35]:
tips = results['response']['tips']['items']

tip = results['response']['tips']['items'][0]
tip.keys()

dict_keys(['id', 'createdAt', 'text', 'type', 'canonicalUrl', 'lang', 'likes', 'logView', 'agreeCount', 'disagreeCount', 'todo', 'user', 'authorInteractionType'])

In [36]:
pd.set_option('display.max_colwidth', -1)

tips_df = json_normalize(tips)

filtered_columns = ['text', 'agreeCount', 'disagreeCount', 'id', 'user.firstName', 'user.lastName', 'user.gender', 'user.id']
tips_filtered = tips_df.loc[:, filtered_columns]

tips_filtered

,text,agreeCount,disagreeCount,id,user.firstName,user.lastName,user.gender,user.id
0,"So amazing, try the cheese grits!",0,0,5c55d5c3418686002cfac954,Caryn,B,NaN,1939393


### Validate Foursquare credentials. 

In [37]:
CLIENT_ID = 'DPR2TRJDTRBYCTN4YIRLNJ4F3GRM3IHAWXWCB4MQGXDU2H5E'
CLIENT_SECRET = 'ZVGOPN1OCEFCGB5RDDUQXBGE11AXDT2OHB0VNGFUAZUO3COZ'
VERSION = '20200713'
LIMIT = 30
print('Your credentials:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET: ' + CLIENT_SECRET)

Your credentials:
CLIENT_ID: DPR2TRJDTRBYCTN4YIRLNJ4F3GRM3IHAWXWCB4MQGXDU2H5E
CLIENT_SECRET: ZVGOPN1OCEFCGB5RDDUQXBGE11AXDT2OHB0VNGFUAZUO3COZ


### Address of potential property in Downtown Corpus Christi.

In [38]:
address = '620 Old Robstown Rd, Corpus Christi, TX'

geolocator = Nominatim(user_agent="Texas_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geographical coordiate of Corpus Christi are {}, {}.'.format(latitude, longitude))

The geographical coordiate of Corpus Christi are 27.79598805516865, -97.42992407545927.


### '620 Old Robstown Rd, Corpus Christi, TX' mark on Folium map of Waco.

In [39]:
map_corpus = folium.Map(location=[latitude, longitude], zoom_start=14)

folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='Downtown Waco Location',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(map_corpus)
 
map_corpus

### First search for "BBQ" in Downtown Corpus Christi.

In [40]:
search_query = 'bbq'
radius = 500
print(search_query + '.... OK!')

bbq.... OK!


In [41]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=DPR2TRJDTRBYCTN4YIRLNJ4F3GRM3IHAWXWCB4MQGXDU2H5E&client_secret=ZVGOPN1OCEFCGB5RDDUQXBGE11AXDT2OHB0VNGFUAZUO3COZ&ll=27.79598805516865,-97.42992407545927&v=20200713&query=bbq&radius=500&limit=30'

In [42]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5f0c7554c740860680bdc8ca'},
 'response': {'venues': []}}

### No locations within 500 meters when search "BBQ".

### Second search for "Barbeque" in Downtown Corpus Christi.

In [43]:
search_query = 'barbeque'
radius = 500
print(search_query + '.... OK!')

barbeque.... OK!


In [44]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=DPR2TRJDTRBYCTN4YIRLNJ4F3GRM3IHAWXWCB4MQGXDU2H5E&client_secret=ZVGOPN1OCEFCGB5RDDUQXBGE11AXDT2OHB0VNGFUAZUO3COZ&ll=27.79598805516865,-97.42992407545927&v=20200713&query=barbeque&radius=500&limit=30'

In [45]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5f0c75c0521bc909eb20f1f7'},
 'response': {'venues': []}}

### No locations within 500 meters when search "Barbeque".